In [1]:
import os
import sys
import statistics

aoc_year, aoc_day = os.getcwd().split(os.sep)[-2:]

# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year {aoc_year} --day {aoc_day}

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 12, 2021...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


\--- Day 12: Passage Pathing ---
----------


*How many paths through this cave system are there that visit small caves at most once?*

In [2]:
from pprint import pprint

with open('input.txt', 'rt') as f:
    lines = [x.strip() for x in f.readlines()]

# Verify parse
pprint(lines[:6])

['yw-MN', 'wn-XB', 'DG-dc', 'MN-wn', 'yw-DG', 'start-dc']


In [3]:
# Find connections
connections = [line.split("-") for line in lines]
# Generate reverse connections
connections += [[c[1],c[0]] for c in connections]
# Drop connections back to start or away from end
connections = [(a,b) for a,b in connections if a != "end" and b != "start"]


In [4]:
def explore(breadcrumbs, connections):
    # Find where we can go from here
    options = [c[1] for c in connections if c[0] == breadcrumbs[-1]]
    # Remove small-cave destinations that have already been visited 
    options = [o for o in options if o.isupper() or (o.islower() and o not in breadcrumbs)]
    #print(f"{breadcrumbs} exploring: {options}")
    if options:
        return [explore(breadcrumbs + [o], connections) for o in options]
    else:    
        return "-->".join(breadcrumbs)

def flatten(paths):
    """Flatten a crazy list rather than debug recursion"""
    while any([isinstance(x, list) for x in paths]):
        paths = [p for p in paths if isinstance(p, str)] + sum([sl for sl in paths if isinstance(sl, list)], [])
    return paths


In [5]:
paths = ["start"]
paths = flatten(explore(paths, connections))
# Reduce to paths that actually end
paths = [p for p in paths if p.endswith("-->end")]

In [6]:
answer1 = len(paths)
print("answer 1:", answer1)

answer 1: 4241


----

In [7]:
# Download part 2
!aoc download --description-only --overwrite --puzzle-file README.md --year {aoc_year} --day {aoc_day}

Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 12, 2021...
Saving puzzle description to "README.md"...
Done!


\--- Part Two ---
----------

After reviewing the available paths, you realize you might have time to visit a single small cave *twice*. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once.

Given these new rules, *how many paths through this cave system are there?*

In [8]:
def explore_expanded(breadcrumbs, connections, depth=99):
    if depth < 0: return "ERROR"
    # Find where we can go from here
    all_options = [c[1] for c in connections if c[0] == breadcrumbs[-1]]
    #print(f"{breadcrumbs} all: {all_options}")
    small_options = [o for o in all_options if o.islower()]
    small_crumbs = [c for c in breadcrumbs if c.islower()]
    has_small_dup = len(small_crumbs) > len(set(small_crumbs))
    #print(f"{breadcrumbs} small: {has_small_dup} {small_options}")

    if has_small_dup:
        # Don't allow another small-cave-duplicate
        small_options = [o for o in small_options if o not in breadcrumbs]
     
    options = [o for o in all_options if o.isupper()] + small_options
    #print(f"{breadcrumbs} exploring: {options}")

    if len(options) > 0:
        return [explore_expanded(breadcrumbs + [o], connections, depth - 1) for o in options]
    else:    
        return "-->".join(breadcrumbs)

In [9]:
paths = ["start"]
paths = flatten(explore_expanded(paths, connections, depth=20))

assert not any(["ERROR" in p for p in paths])
# Reduce to paths that actually end
paths = [p for p in paths if p.endswith("-->end")]

In [10]:
answer2 = len(paths)
print("answer 2:", answer2)

answer 2: 122134
